In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random
import datetime
import os
cwd = Path(os.getcwd())

In [ ]:
'''
Function used to standardise input dates.  Without this the datatype 
sometime includes timestamps.

Some inputs use columns with 'Date', or 'StatusDate', or other weird things
'''
def convertDate(row):
    for k in row.keys():
        if 'date' in k.lower():
            row['date'] = row[k].date()
    return row


'''
These are the ordinary differential equations that SciPy's odeint() will solve
Beta is a float from 0-1 and describes how infectious the virus is
Gamma is a float from 0-1 and describes the recovery period.  Best thought of 
as 1 / (recovery days)  
'''
def deriv(y, t, beta, gamma, P0):
    S, I, R = y
    dSdt = -beta * S * I / P0
    dIdt = beta * S * I / P0 - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt


In [ ]:
df = pd.read_excel(cwd / 'Inputs.xlsx')
df = df.apply(lambda x: convertDate(x), axis = 1)

targets = pd.read_excel(cwd / 'Forecase.xlsx')
targets = targets.dropna()

In [ ]:
actualEndDate = datetime.datetime.strptime('2022-01-25','%Y-%m-%d').date() #when the prediction will end

In [ ]:
endDay = datetime.datetime.strptime('2022-05-01','%Y-%m-%d').date()

result = []

for hospital in targets['HHS'].unique():
    print(hospital)
    
    # each HHS starts its curve at a different time, this grabs that value
    startDate = pd.Timestamp(targets[(targets['HHS'] == hospital)]['start'].values[0]).date()
    simDays = (endDay - startDate).days
    t = np.linspace(0, simDays, simDays)    
    
    values = df[((df['date'] >= startDate) & (df['date'] <= actualEndDate) & (df['Area'] == hospital))]['Active_Inpatients'].to_numpy()
    for risk in ['likely', 'worst']:
        
        # It was suggested to have approximately 85% of the population recovered by the end of the wave.   
        P0 = targets[(targets['HHS'] == hospital)][risk].values[0] / 0.85
        # Initial number of infected and recovered individuals, I0 and R0.
        I0, R0 = values[0], 0.
        
        # Everyone else, S0, is susceptible to infection initially.
        S0 = P0 - I0 - R0 
        y0 = [S0, I0, R0]   
        # We have been asked to test for different recovery 
        # periods from 2 - 10 days in increments of 0.5 days)
        for recoveryRate in np.arange(2,11,0.5):
            
            minError = 1e50  # some big number to make sure at least 1 number is chosen
  
            # Could probably do something smart with early stopping 
            # where the error gets smaller, then bigger.  But i haven't
            # done that
            for infectionRate in np.arange(0,1,1/stepSize):
                ret = odeint(deriv, y0, t, args=(infectionRate, 1/recoveryRate, P0))
                S, I, R = ret.T
                
                # Some of squared error
                error = np.sum((values - I[:len(values)])**2) 
                if error < minError:
                    minError = error
                    goodVal = infectionRate
                    goodI = I
                    goodR = R[-1]      
                    
            # activeCases is the sum of the last recoveryRate days worth of newCases
            # Therefore on a given day activeCases_t = sum from t-recoveryRate to t of newCases_t
            # activeCases_(t) - activeCases_(t-1) = newCases_t - newCases_(t-recoveryRate-1)
            
            newCases = [goodI[0]/recoveryRate,] * int(recoveryRate)
            for idx, i in enumerate(goodI):
                if idx > 0:
                    #because we have preloaded newCases to be of length recoveryRate we can just take the idx'th value here    
                    nC = goodI[idx] - goodI[idx - 1] + newCases[idx]
                    nC = np.max([0, nC])
                else:
                    nC = newCases[idx]            
                newCases.append(nC)
                
                # Originally this is where the code ended, so I built the DataFrame here
                # that has now changed, but it is easier to use the DataFrame...so I keep
                # this in.
                result.append({
                    "date": startDate + datetime.timedelta(days = idx),
                    "admissions" : float(f"{np.max([0,i]):0.2f}"),
                    "newCases" : float(f"{nC:0.2f}"),
                    "hhs" : hospital,
                    "type" : risk,
                    "recoveryRate" : float(f"{recoveryRate:0.1f}"),
                    "recovered" : float(f"{goodR:0.1f}"),
                    "infectionRate" : float(f"{goodVal:0.3f}")
                })
                
tempDF = pd.DataFrame(result)    
1

In [ ]:
'''
Use the previously made df to find the average infectionRate for 
each recoveryRate / risk combination
'''
result = []
for risk in ['likely', 'worst']:
    for recoveryRate in tempDF['recoveryRate'].unique():
        avg = tempDF[((tempDF['type'] == risk) & (tempDF['recoveryRate'] == recoveryRate))]
        avg = avg.groupby('hhs')['infectionRate']
        avg = avg.unique()    
        
        # Calculate centre mean - excluding max and min number
        avg = (avg.sum() - avg.max() - avg.min()) / (len(avg) - 2)
        
        result.append(
        {
            "type" : risk,
            "infectionRate" : avg[0],
            "recoveryRate" : recoveryRate
        })
avgInfectDF = pd.DataFrame(result)
1

In [ ]:
'''
This method is used to fit the average infectionRate curve to actual data for each HHS.
I really thought i could use np.correlate to do this, but it gave inconsistent results.

Surely there is room to improve this!
'''

def getModelStart(values, I):
    values = np.array(values)
    error = []
    maxChange = 15
    maxRange = np.min([maxChange*2+1, len(values) * 2 + 1 -2])
    maxChange = (maxRange - 1) // 2
    for idx in range(maxRange):
        magicNumber = np.abs(idx - maxChange)
        if idx <= maxChange:
            arrayA = I[magicNumber: len(values) + magicNumber]
            arrayB = values
        elif idx <= (2*maxChange):
            arrayA = I[:len(values[magicNumber:])]
            arrayB = values[magicNumber:]
        else:
            raise ValueError("Busted it!")
        assert len(arrayA) == len(arrayB), "things should be the same length!!!"

        # normally i would use sum of squared error; however, the length of 
        # compared values is not consistent.  Therefore, I went with mean
        error.append(np.mean((arrayA - arrayB)**2)) 
    return int(np.argmin(error) - maxChange)

In [ ]:
'''
Yes, this is very similar to the code above.  The old difference is we only 
doing 1 infectionRate
'''

endDay = datetime.datetime.strptime('2022-05-01','%Y-%m-%d').date()
result = []
for hospital in targets['HHS'].unique():
    startDate = pd.Timestamp(targets[(targets['HHS'] == hospital)]['start'].values[0]).date()
    simDays = (endDay - startDate).days
    t = np.linspace(0, simDays, simDays)        
    
    values = df[((df['date'] >= startDate) & (df['Area'] == hospital))]['Active_Inpatients'].to_numpy()#[::-1]
    for risk in ['likely', 'worst']:
        
        # It was suggested to have approximately 85% of the population recovered by the end of the wave.   
        P0 = targets[(targets['HHS'] == hospital)][risk].values[0] / 0.85
        # Initial number of infected and recovered individuals, I0 and R0.
        I0, R0 = values[0], 0.
        
        # Everyone else, S0, is susceptible to infection initially.
        S0 = P0 - I0 - R0 
        y0 = [S0, I0, R0]          
        for recoveryRate in np.arange(2,11,0.5):
            infectionRate = avgInfectDF[((avgInfectDF['type'] == risk) & (avgInfectDF['recoveryRate'] == recoveryRate))]['infectionRate'].values[0]
            ret = odeint(deriv, y0, t, args=(infectionRate, 1/recoveryRate, P0))
            S, I, R = ret.T
            
            # I guessed the start date for each HHS, now with the new 
            # curve it is possible they start earlier/later.  This is 
            # where we make that adjustment
            changeStart = getModelStart(values, I)
            modelStart = startDate + datetime.timedelta(days = changeStart)            
            
            newCases = [I[0] / recoveryRate,] * int(recoveryRate)
            for idx, i in enumerate(I):
                if idx > 0:
                    #because we have preloaded newCases to be of length recoveryRate we can just take the idx'th value here    
                    nC = I[idx] - I[idx - 1] + newCases[idx]
                    nC = np.max([0, nC])
                else:
                    nC = newCases[idx]            
                newCases.append(nC)
                result.append({
                    "date": modelStart + datetime.timedelta(days = idx),
                    "admissions" : float(f"{np.max([0,i]):0.2f}"),
                    "newCases" : float(f"{nC:0.2f}"),
                    "hhs" : hospital,
                    "type" : risk,
                    "recoveryRate" : float(f"{recoveryRate:0.1f}")
                })            
            
rdf = pd.DataFrame(result)
1
            

In [ ]:

today = datetime.datetime.today()

rdf.to_excel(cwd / f"{today:%y%m%d}-Output-Hospital.xlsx", index = False)

In [ ]:
'''
Makes a pretty picture.  I've turned off x and y axis values because everything is too 
cramped.  If you want to know the exact values then a specific query should be run.
'''
predLength = 90
recoveryRate = 4
# Set facecolor to white because I use a dark theme and cannot read anything otherwise
fix,axs = plt.subplots(nrows=4, ncols=3, facecolor='white', figsize=(10,10))

for idx, hospital in enumerate(rdf['hhs'].unique()):
    
    startDate = pd.Timestamp(targets[(targets['HHS'] == hospital)]['start'].values[0]).date()
    values = df[((df['date'] >= startDate) & (df['Area'] == hospital))]['Active_Inpatients'].to_numpy()
    days = []
    for i,_ in enumerate(values):
        days.append(startDate + datetime.timedelta(days = i))

    axs[idx//3, idx%3].plot(days,values, c='b', label = 'Actuals')
    y = rdf[((rdf['hhs'] == hospital) & (rdf['type'] == 'likely') & (rdf['recoveryRate'] == recoveryRate))]['admissions']
    changeStart = getModelStart(values, y)
    modelStart = startDate + datetime.timedelta(days = changeStart) 
    
    predDays = []
    for i, _ in enumerate(y):
        predDays.append(modelStart + datetime.timedelta(days = i))

    y = y[:predLength]
    predDays = predDays[:predLength]
    print(f"{predDays[-1]} {hospital}")
    axs[idx//3, idx%3].plot(predDays, y, c='g',label = 'Prediction')
    
    today = datetime.datetime.today().date() - datetime.timedelta(days = 1) 
    axs[idx//3, idx%3].plot([actualEndDate, actualEndDate], [y.min(), y.max()], c='k', alpha = 0.5)
    axs[idx//3, idx%3].set_xticks([])
    axs[idx//3, idx%3].set_yticks([])
    axs[idx//3, idx%3].title.set_text(f'{hospital}')
    
startDate = pd.Timestamp(targets['start'].min()).date()
values = df.groupby('date')['Active_Inpatients'].sum().to_numpy()
days = []
for i,_ in enumerate(values):
    days.append(startDate + datetime.timedelta(days = i))
    
axs[3, 2].plot(days,values, c='b', label = 'Actuals')
y = rdf[( (rdf['type'] == 'likely') & (rdf['recoveryRate'] == recoveryRate))].groupby('date')['admissions'].sum().to_numpy()
changeStart = getModelStart(values, y)
modelStart = startDate + datetime.timedelta(days = changeStart) 

predDays = []
for i, _ in enumerate(y):
    predDays.append(modelStart + datetime.timedelta(days = i))

y = y[:predLength]
predDays = predDays[:predLength]
print(f"{predDays[-1]} {hospital}")
axs[3, 2].plot(predDays, y, c='g',label = 'Prediction')

today = datetime.datetime.today().date() - datetime.timedelta(days = 1) 
axs[3, 2].plot([actualEndDate, actualEndDate], [y.min(), y.max()], c='k', alpha = 0.5)
axs[3, 2].set_xticks([])
axs[3, 2].set_yticks([])
axs[3, 2].title.set_text(f'Total')    